In [ ]:
import requests
import re

# Cipher text for analysis
cipher_text = "PRCSOFQX FP QDR AFOPQ CZSPR LA JFPALOQSKR. QDFP FP ZK LIU"

In [ ]:
# Retrieve and prepare the dictionary
dict_url = "https://raw.githubusercontent.com/dwyl/english-words/master/words.txt"
dictionary = requests.get(dict_url).text.split("\n")
# Display sample dictionary items
print(dictionary[100:120])

In [ ]:
# Analyze the dictionary by word length
word_lengths = [2, 3, 4, 5, 6, 7]
for length in word_lengths:
    count = len([word for word in dictionary if len(word) == length])
    print(f"Length {length}: ", count)

In [ ]:
def rearrange_by_score_and_length(words):
    frequencies = {}
    word_scores = {}

    unique_words = set(words)
    # Calculate character frequencies
    for word in unique_words:
        for char in word:
            frequencies[char] = frequencies.get(char, 0) + 1
    # Calculate word scores based on character frequencies
    for word in unique_words:
        score = sum([frequencies[char] for char in word])
        word_scores[word] = score

    # Sort words by length and score
    sorted_words = sorted(unique_words, key=lambda word: -len(word) * word_scores[word])

    for word in sorted_words:
        print(word, len(word), word_scores[word], len(word) / word_scores[word])
    return sorted_words


sorted_cipher_words = rearrange_by_score_and_length(cipher_text)
print(sorted_cipher_words)

In [ ]:
def get_translation(mapping, text):
    return "".join([mapping[char] if char in mapping else char for char in text])


def create_regex_pattern(word):
    return f'^{".".join(["." if char.isupper() else char for char in word])}$'


def search_matching_words_in_dict(pattern, word_list):
    compiled_pattern = re.compile(pattern)
    return [word for word in word_list if compiled_pattern.match(word)]


def exclude_known_translations(known_vals, candidate_words):
    return [
        word
        for word in candidate_words
        if all([char not in known_vals for char in word])
    ]


def apply_translation(mapping, encrypted, decrypted):
    for i, char in enumerate(encrypted):
        if char.isupper():
            if decrypted[i] in mapping.values():
                return False
            mapping[char] = decrypted[i]
    return True


def brute_force_decrypt(mapping, cipher_words, word_list):
    if not cipher_words:
        print("Potential solution:", mapping)
        return [mapping]
    
    current_word = cipher_words[0]
    translated_word = get_translation(mapping, current_word)
    
    if translated_word.islower():
        return brute_force_decrypt(mapping, cipher_words[1:], word_list)
    
    regex_pattern = create_regex_pattern(translated_word)
    matching_words = search_matching_words_in_dict(regex_pattern, word_list)
    
    known_translation_chars = [char for char in translated_word if char.islower()]
    unknown_translation_values = [val for val in mapping.values() if val not in known_translation_chars]
    filtered_words = exclude_known_translations(unknown_translation_values, matching_words)
    
    print(f"Potential matches for {current_word}: {', '.join(filtered_words[:8])}")
    print("Translation so far:", get_translation(mapping, " ".join(cipher_text)))
    
    potential_solutions = []
    for word in filtered_words:
        new_mapping = mapping.copy()
        if apply_translation(new_mapping, translated_word, word):
            found_solutions = brute_force_decrypt(new_mapping, cipher_words[1:], word_list)
            potential_solutions.extend(found_solutions)
    return potential_solutions

In [ ]:
found_translations = brute_force_decrypt({}, sorted_cipher_words, dictionary)
print(found_translations)